Original dataset:

https://earth-info.nga.mil/php/download.php?file=egm-08spherical

In [1]:
# Load the string representations of the C/S
# coefficients up to the maximum degree into
# a list of tuples.
max_degree=80
CS_list = []
with open("EGM2008_to2190_TideFree", "r") as f:
    for line in f:
        fields = line.split()
        cur_deg = int(fields[0])
        if cur_deg > max_degree:
            break
        # NOTE: replace 'D' with 'e' for parsing
        # of doubles in C++.
        cur_C = fields[2].replace('D','e')
        cur_S = fields[3].replace('D','e')
        CS_list.append((cur_C, cur_S))

# Explanation: for a degree n, there are n+1 orders.
# Hence, at degree n we expect a total of (n+1)*(n+2)/2
# coefficients, minus 3 because the first coefficient order
# is 2 and not 0.
assert len(CS_list) == ((max_degree+1)*(max_degree + 2)) // 2 - 3

In [2]:
# Template for the header file output.
HPP_FILE_TMPLT = r"""#ifndef HEYOKA_DETAIL_EGM2008_HPP
#define HEYOKA_DETAIL_EGM2008_HPP

#include <cstdint>

#include <heyoka/config.hpp>

HEYOKA_BEGIN_NAMESPACE

namespace model::detail
{{

inline constexpr std::uint32_t egm2008_max_degree = {0};

extern const double egm2008_CS[{1}][2];

}} // namespace model::detail

HEYOKA_END_NAMESPACE

#endif

"""

# Template for the cpp file output.
CPP_FILE_TMPLT = r"""#include <heyoka/config.hpp>
#include <heyoka/detail/egm2008.hpp>

HEYOKA_BEGIN_NAMESPACE

namespace model::detail
{{

const double egm2008_CS[{0}][2] = {1};

}} // namespace model::detail

HEYOKA_END_NAMESPACE

"""

In [3]:
# Write the files.
with open('egm2008.hpp', 'w') as f:
    f.write(HPP_FILE_TMPLT.format(max_degree, len(CS_list)))

with open('egm2008.cpp', 'w') as f:
    f.write(CPP_FILE_TMPLT.format(len(CS_list), f'{{{',\n'.join(f'{{{_[0]},{_[1]}}}' for _ in CS_list)}}}'))

## Adapting the recursion formulae

The original recursion formulae for $V$ are:

\begin{align}
V_{nm} & = \left(\frac{2n-1}{n-m}\right)\cdot\frac{za}{r^2}\cdot V_{n-1,m} - \left(\frac{n+m-1}{n-m}\right)\cdot\frac{a^2}{r^2}\cdot V_{n-2,m},\\
V_{mm} & = \left(2m-1\right)\left[ \frac{xa}{r^2}V_{m-1,m-1} - \frac{ya}{r^2}W_{m-1,m-1} \right]
\end{align}

(Montenbruck 3.2.4). The formulae for $W$ are identical (modulo a sign change). The potential is then given by:

$$
U = \frac{GM}{a}\sum_{n=0}^\infty\sum_{m=0}^n\left(C_{nm}V_{nm}+S_{nm}W_{nm}\right).
$$

The EGM2008 model does not give directly $C_{nm}$ and $S_{nm}$ but rather their normalised counterparts

\begin{align}
\overline{C}_{nm} & = \alpha_{nm}C_{nm},\\
\overline{S}_{nm} & = \alpha_{nm}S_{nm},
\end{align}

where

$$
\alpha_{nm}=\sqrt{\frac{\left( n+m \right)!}{\left( 2-\delta_{0m}\right)\left(2n+1\right)\left( n-m \right)!}}.
$$

We can thus define

\begin{align}
\overline{V}_{nm} & = \frac{V_{nm}}{\alpha_{nm}},\\
\overline{W}_{nm} & = \frac{W_{nm}}{\alpha_{nm}},
\end{align}

and rewrite the potential in terms of the normalised coefficients as

$$
U = \frac{GM}{a}\sum_{n=0}^\infty\sum_{m=0}^n\left(\overline{C}_{nm}\overline{V}_{nm}+\overline{S}_{nm}\overline{W}_{nm}\right).
$$

We then need to formulate the recursion formulae for $\overline{V}$ and $\overline{W}$ accounting for the normalisation coefficient $\alpha_{nm}$. The end result is:

\begin{align}
\overline{V}_{nm} & = \sqrt{\frac{\left( 2n+1 \right)\left( 2n-1 \right)}{\left( n-m \right) \left( n+m \right)}}\cdot\frac{za}{r^2}\cdot\overline{V}_{n-1,m}-\sqrt{\frac{\left( 2n+1 \right)\left( n-m-1 \right)\left( n+m-1 \right)}{\left( n-m \right)\left( n+m\right)\left( 2n-3\right)}}\cdot\frac{a^2}{r^2}\cdot\overline{V}_{n-2,m},\\
\overline{V}_{mm} & = \sqrt{\frac{\left(2-\delta_{0m} \right) \left( 2m+1 \right)}{2m\left( 2-\delta_{0,m-1}\right)}}\left[ \frac{xa}{r^2}\cdot\overline{V}_{m-1,m-1} - \frac{ya}{r^2}\cdot\overline{W}_{m-1,m-1 }\right]
\end{align}
(and similarly for $W$, apart from a sign change).

## Code to check the implementation

This is the code which uses [pyshtools](https://shtools.github.io/SHTOOLS/) to generate the comparison data used in the C++ test.

In [1]:
# Deterministic random seeding.
import numpy as np
rng = np.random.default_rng(12345)

# Pick random points around the earth.
nsamples = 500
u = rng.uniform(size=(nsamples,))
v = rng.uniform(size=(nsamples,))
phi = np.pi*2*u
theta = np.arccos(2*v-1)
r = rng.uniform(low=6378137.0, high=2*6378137.0, size=(nsamples,))

# Convert the positions to cartesian.
x = r*np.sin(theta)*np.cos(phi)
y = r*np.sin(theta)*np.sin(phi)
z = r*np.cos(theta)
positions = np.array([x,y,z]).T

# Compute the gravitational acceleration according to pyshtools.
import pyshtools
coeffs = pyshtools.datasets.Earth.EGM2008(30)
# NOTE: setting omega to zero is important, otherwise
# pyshtools accounts for the Earth's rotation when computing
# the acceleration.
coeffs.set_omega(0.)
acc = coeffs.expand(colat=theta,lon=phi,r=r,degrees=False)

# NOTE: pyshtools returns the acceleration components wrt spherical unit
# vectors. We need to apply a transformation in order to get the components
# wrt the Cartesian unit vectors. See:
#
# https://en.wikipedia.org/wiki/Vector_fields_in_cylindrical_and_spherical_coordinates
cart_acc = []
for i in range(nsamples):
    cur_theta = theta[i]
    cur_phi = phi[i]
    cur_r = r[i]

    # Build the rotation matrix.
    rot = np.array([[np.sin(cur_theta)*np.cos(cur_phi),
                     np.cos(cur_theta)*np.cos(cur_phi),
                     -np.sin(cur_phi)],
                    [np.sin(cur_theta)*np.sin(cur_phi),
                     np.cos(cur_theta)*np.sin(cur_phi),
                     np.cos(cur_phi)],
                    [np.cos(cur_theta),-np.sin(cur_theta),0.]])

    # Apply the rotation.
    cart_acc.append(rot @ acc[i])

cart_acc = np.array(cart_acc)

In [ ]:
# Print with full precision.
import sys
np.set_printoptions(threshold=sys.maxsize, precision=17)
cart_acc